# Turn exportable Bixi data (pdf) into usable csv format

In [24]:
# import dependencies
import pandas as pd
import numpy as np
import csv
import tabula
import editdistance

In [25]:
# Create dictionary for station names to station codes
with open('../data/bixi/BixiMontrealRentals2018/Stations_2018.csv', mode='r') as f_in:
    reader = csv.reader(f_in)
    station_dict = {rows[1]:rows[0] for rows in reader}

In [26]:
# Set username and import data file (csv). Use https://www.zamzar.com/convert/pdf-to-csv/ to create csv files
username = 'saad'
df_master = pd.read_csv('../data/bixi/%s.csv'%username,names = ["timestamp",'location','duration'],skiprows=7)

In [27]:
# Keep all station_points
df = df_master.drop(df_master[~(df_master['timestamp'].str.startswith('Start') | df_master['timestamp'].str.startswith('End'))].index)
df.reset_index(drop = True, inplace = True)
df.rename(columns = {'timestamp':'date'}, inplace = True)
df.head()


,date,location,duration
0,Start: 11/15/2018,Duluth / St-Laurent,11 min 10 s
1,End: 11/15/2018,de la Montagne / Sherbrooke,NaN
2,Start: 11/15/2018,Milton / University,8 min 23 s
3,End: 11/15/2018,Duluth / St-Laurent,NaN
4,Start: 11/15/2018,Mackay / de Maisonneuve,5 min 51 s


In [28]:
# Remove all NaNs from the start/end stations and including its trip-partner (end/start)
invalid_index = df['location'].isna()
for index, row in invalid_index.iteritems():
    if row:
        if np.mod(index,2) == 1:
            invalid_index.loc[index-1] = True
        elif np.mod(index,2) == 0:
            invalid_index.loc[index+1] = True
            
df.drop(df[invalid_index].index,inplace=True)

In [29]:
# For those entries that do not match any key in the dictionary, compute Levenshtein distance for all keys. Pick the smallest
for index, item in df['location'].iteritems():
    if item in station_dict.keys():
        continue
    else:
        min_dist = 100
        for station_name in station_dict.keys():
            dist = editdistance.eval(item, station_name)
            if dist < min_dist:
                min_dist = dist
                min_station = station_name
        df.loc[index,'location'] = min_station

In [30]:
# Create empty dataframe with headers corresponding to known format
with open('../data/bixi/BixiMontrealRentals2018/OD_2018-04.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)

df_full = pd.DataFrame(columns = header)

In [31]:
# Place the start/end stations and duration in df_full
for index, row in df.iterrows():
    if np.mod(index,2) == 0:
        df_full.loc[int(index/2),'start_station_code'] = station_dict[row[1]]
        df_full.loc[int(index/2),'duration_sec'] = 60*int(row[2].split()[0])+int(row[2].split()[2])
    elif np.mod(index,2) == 1:
        df_full.loc[int(index/2),'end_station_code'] = station_dict[row[1]]


In [32]:
print(df_full)

    start_date start_station_code end_date end_station_code duration_sec  \
0          NaN               6213      NaN             6065          670   
1          NaN               6070      NaN             6213          503   
2          NaN               6100      NaN             7080          351   
3          NaN               6432      NaN             6100          527   
4          NaN               6065      NaN             6432          545   
5          NaN               6344      NaN             6065         1628   
6          NaN               6344      NaN             6344          399   
7          NaN               6205      NaN             6065          627   
8          NaN               6070      NaN             6213          473   
9          NaN               6205      NaN             6065          445   
10         NaN               6070      NaN             6205          147   
11         NaN               6070      NaN             6081          362   
12         N

In [76]:
# and save the file
df_full.to_csv('../data/bixi/%s-complete.csv'%username,index=False)

In [31]:
'''
Adding the time is too time-consuming (get it?). There are more important jobs to do. Surely one can find to robustly add the time to the userprofile

df_time = df_master.drop(df_master[~(df_master['timestamp'].str.contains('AM') | df_master['timestamp'].str.contains('PM'))].index)
df_time.reset_index(drop = True, inplace = True)
df_time.rename(columns = {'timestamp':'time'}, inplace = True)

''';

In [32]:
df_time[~df_time['location'].isna()];

In [ ]:
'''Full empty dataframe with 
1. start_date & end_date (date + time)
2. start_station_code & end_station_code
3. duration-sec
